# Final visualisation using Bokeh

In [ ]:
import pandas as pd
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Slider, CategoricalColorMapper, HoverTool, Select
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import show, curdoc, output_notebook
from bokeh.layouts import row, widgetbox
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

# This allows the final Bokeh visualisation to be run within the notebook.
output_notebook()

In [ ]:
# This adds the political alignment data to the final, merged dataframe (df)
df = pd.read_csv('./data/final_merge.csv')
df2 = pd.read_csv('./data/us_state_alignment.csv')
df2 = df2[df2['Year'] >= 1980]
df2 = df2[df2['Year'] <= 2014]
df = df.set_index(['State', 'Year'])
df = df.sort_index()
df = df.reset_index()
alignment = df2['Alignment']
alignment = alignment.reset_index()
alignment = alignment.drop(['index'], axis=1)
df['Alignment'] = alignment
df = df.drop(['StateYear'], axis=1)
df = df.set_index('Year')
df['Total domestic water use per capita, gallons/person/day'].fillna(df['Total domestic water use per capita, gallons/person/day'].mean(),inplace=True)
df['Average ranking'].fillna(df['Average ranking'].mean(), inplace=True)
df = df.replace({'R': 'Republican', 'D': 'Democratic'})
df = df.rename(columns={'Average ranking':'Average ranking across factors',
           'Total energy consumption ranking':'Ranking for total energy consumption',
           'Renewable energy consumption ranking':'Ranking for renewable energy consumption',
           'Petroleum product consumption ranking':'Ranking for petroleum product consumption',
           'Residential co2 emissions, metric tons/person':'Ranking for per capita domestic CO2 emissions',
           'Total domestic water use per capita, gallons/person/day': 'Ranking for per capita domestic water use'})
df.head()

In [ ]:
def modify_doc(doc) :   
    # Colour mapper for plot and legend
    color_mapper = CategoricalColorMapper(factors=['Democratic', 'Republican'],
                                          palette=['blue', 'red'])
    # ColumnDataSource
    source = ColumnDataSource(data={
        'x': df['Alignment'].loc[1980],
        'y': df['Average ranking across factors'].loc[1980],
        'state': df['State'].loc[1980],
        'election_outcome': df['Outcome'].loc[1980],
    })
    
    # Save the minimum and maximum values of the fertility column: xmin, xmax
    xmin, xmax = min(df['Alignment']), max(df['Alignment'])

    # Save the minimum and maximum values of the life expectancy column: ymin, ymax
    ymin, ymax = min(df['Average ranking across factors']), max(df['Average ranking across factors'])

    # Create the figure: plot
    plot = figure(title='Political alignment vs average ranking across factors (1980)')

    # Add circle glyphs to the plot
    plot.circle('x', 'y', fill_alpha=0.8, source=source,
               color=dict(field='election_outcome', transform=color_mapper), legend='election_outcome')

    # Set the legend.location attribute of the plot to 'top_right'
    plot.legend.location = 'top_right'

    # Set the x-axis label
    plot.xaxis.axis_label ='Political alignment'

    # Set the y-axis label
    plot.yaxis.axis_label = 'Average ranking across all factors'
    
    # Define the callback function: update_plot
    def update_plot(attr, old, new):
    # set the `yr` name to `slider.value` and `source.data = new_data`
        yr = slider.value
        y = y_select.value
        
        plot.yaxis.axis_label = y
        
        new_data = {
            'x': df['Alignment'].loc[yr],
            'y': df[y].loc[yr],
            'state': df['State'].loc[yr],
            'election_outcome': df['Outcome'].loc[yr],
        }
        source.data = new_data
        
        # Set the range of all axes
        plot.y_range.start = min(df[y])
        plot.y_range.end = max(df[y])
            
        # Add title to figure: plot.title.text
        plot.title.text = 'Political alignment vs %s (%d)' % (y.lower(), yr)

    # Make a slider object: slider
    slider = Slider(start=1980, end=2014, step=1, value=1980, title='Year')

    # Attach the callback to the 'value' property of slider
    slider.on_change('value', update_plot)
    
    # Create a dropdown Select widget for the y data: y_select
    y_select = Select(options=[('Average ranking across factors', 'Average across all factors'),
                              ('Ranking for total energy consumption', 'Total energy consumption'),
                              ('Ranking for renewable energy consumption', 'Renewable energy consumption'),
                              ('Ranking for petroleum product consumption', 'Petroleum product consumption'),
                              ('Ranking for per capita domestic CO2 emissions', 'Per capita domestic CO2 emissions'),
                              ('Ranking for per capita domestic water use', 'Per capita domestic water use')],
                      value='Average ranking across factors',
                      title='Select a ranking:')

    # Attach the update_plot callback to the 'value' property of y_select
    y_select.on_change('value', update_plot)
    
    # Create a HoverTool: hover
    hover = HoverTool(tooltips=[('State', '@state'),
                                ('Political alignment', '@x'),
                               ('Ranking', '@y')])

    # Add the HoverTool to the plot
    plot.add_tools(hover)

    # Make a row layout of widgetbox(slider) and plot and add it to the current document
    layout = row(widgetbox(slider, y_select), plot)

    # Make a row layout of widgetbox(slider) and plot and add it to the current document
    doc.add_root(layout)

# Set up the Application 
handler = FunctionHandler(modify_doc)
app = Application(handler)

show(app, notebook_url="localhost:8888")